<a href="https://colab.research.google.com/github/HomayounfarM/Image-processing/blob/main/Edge_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from google.colab import drive
import os

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
import skimage
from skimage.filters import median
from skimage.morphology import disk
from skimage import feature
from scipy.ndimage import distance_transform_edt
from skimage import measure
import cv2 as cv
import cv2 as cv2
from skimage.segmentation import watershed
from scipy import ndimage as ndi
from sklearn.cluster import KMeans
import skimage.io
from skimage import data

In [ ]:
from google.colab import drive
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image


# Increase the allowed pixel limit (use a value that suits your needs)
Image.MAX_IMAGE_PIXELS = None  # Setting to None removes the limit

# Mount Google Drive
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/Colab Notebooks/Data/image processing/')

img = plt.imread('20221115_LaBelle_2D_21MB_150ft_8ftpersec_transparent_mosaic_group1.tif')
#img = cv.imread(r'A_1_DR_Labelle Tomato Image.tif')
plt.imshow(img)


In [ ]:
cropped_image = img[2000:4000, 7800:8500]
cropped_image.shape
#plt.figure(figsize=(10,20))
#plt.imshow(cropped_image)

In [ ]:
# OpenCV by default reads images in BGR format. You can use the cvtColor(image, flag)
# and the flag we looked at above to fix this:

cropped_image_RGB = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
cropped_image_hsv = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)

f, (ax0,ax1,ax2) = plt.subplots(1,3, figsize=(10,500))

ax0.imshow(cropped_image)
ax1.imshow(cropped_image_RGB)
ax2.imshow(cropped_image_hsv)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors

In [ ]:
r, g, b = cv2.split(cropped_image_RGB)
fig = plt.figure(figsize=(10,10))
axis = fig.add_subplot(1, 1, 1, projection="3d")

pixel_colors = cropped_image_RGB.reshape((np.shape(cropped_image_RGB)[0]*np.shape(cropped_image_RGB)[1], 3))
norm = colors.Normalize(vmin=-1.,vmax=1.)
norm.autoscale(pixel_colors)
pixel_colors = norm(pixel_colors).tolist()

axis.scatter(r.flatten(), g.flatten(), b.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Red")
axis.set_ylabel("Green")
axis.set_zlabel("Blue")
plt.show()

In [ ]:
h, s, v = cv2.split(cropped_image_hsv)
fig = plt.figure(figsize=(10,10))
axis = fig.add_subplot(1, 1, 1, projection="3d")

pixel_colors = cropped_image_hsv.reshape((np.shape(cropped_image_hsv)[0]*np.shape(cropped_image_hsv)[1], 3))
norm = colors.Normalize(vmin=-1.,vmax=1.)
norm.autoscale(pixel_colors)
pixel_colors = norm(pixel_colors).tolist()

axis.scatter(h.flatten(), s.flatten(), v.flatten(), facecolors=pixel_colors, marker=".")
axis.set_xlabel("Hue")
axis.set_ylabel("Saturation")
axis.set_zlabel("Value")
plt.show()

In [ ]:
denoised = cv.medianBlur(cropped_image,5)
hsv = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
f, (ax0,ax1,ax2,ax3)=plt.subplots(1,4, figsize=(10,20))
ax0.imshow(cropped_image)
ax1.imshow(denoised)
ax2.imshow(hsv)

lower_green = np.array([25, 40, 40])
upper_green = np.array([80, 255, 255])
mask1 = cv2.inRange(hsv, lower_green, upper_green)                          # Using threshold to male a mask
result_thresh = cv2.bitwise_and(cropped_image, cropped_image, mask=mask1)   # to show the mask on the original image

ax3.imshow(mask1)

In [ ]:
cropped_image_gray = cv.cvtColor(cropped_image, cv.COLOR_BGR2GRAY)
cropped_image_den_gray = cv.cvtColor(denoised, cv.COLOR_BGR2GRAY)
cropped_image_hsv_gray = cv.cvtColor(hsv, cv.COLOR_BGR2GRAY)

h, s, v = cv2.split(hsv)

plt.figure(figsize=(10,20))
plt.imshow(h)

In [ ]:
plt.hist(h, bins=5)

In [ ]:
from sklearn.cluster import KMeans
KMeans=KMeans(n_clusters=3)
h1=np.array(h).reshape(-1,1)
KMeans.fit(np.array(h1))
print(KMeans.cluster_centers_)

In [ ]:
# Get cluster assignments for each data point
cluster_assignments = KMeans.labels_
cluster_assignments = cluster_assignments.reshape(2000,700)
cluster_assignments[cluster_assignments==2]=0
# Get the coordinates of the cluster centers
cluster_centers = KMeans.cluster_centers_

cluster_assignments[cluster_assignments==1]=255

# Plot the data points and cluster centers
f, (ax0, ax1, ax2, ax3) = plt.subplots(1,4, figsize=(10,20))
ax0.imshow(cluster_assignments)
ax1.imshow(cropped_image)
ax2.imshow(mask1)
ax3.imshow(h)